# 1 | Preparation
Import necessary libraries.

In [13]:
import numpy as np
import pandas as pd

# 2 | Privacy Protection
This dataset contains values that reveals specific individual and institution name. To protect their identity, I used anonymization technique (enumeration) to replace every individual names. As for the specific institution name mentioned, I simply delete them but not to the point that the necessary data is eradicated. As a side note, since this dataset is stored on my public repo, I have censored the individual names beforehand, before it even staged for preparation.

In [15]:
df = pd.read_csv('data.csv', sep=';')
df['Nama Lengkap'] = [f'Student {i+1}' for i in range(len(df))]
df = df.rename(columns={'Lulus dari SPENSANA  ...': 'Lulus dari SMP',
                       'Jurusan ...': 'Jurusan'})
df

,Timestamp,Nama Lengkap,Jenis Kelamin,Lulus dari SMP,Lanjut ke Sekolah Menengah ...,Jurusan
0,8/25/2025 9:45:19,Student 1,Laki-laki,2025,SMK,Agribisnis Tanaman Pangan Dan Hortikultura (ATPH)
1,8/25/2025 9:51:54,Student 2,Laki-laki,2024-2025,SMA,IPS
2,8/25/2025 9:57:09,Student 3,Laki-laki,2025,SMK,Teknik Otomasi Industri
3,8/25/2025 10:05:45,Student 4,Laki-laki,2024-2025,SMK,Teknik kendaraan ringan
4,8/25/2025 10:08:43,Student 5,Perempuan,2025,SMK,USAHA LAYANAN PARIWISATA
...,...,...,...,...,...,...
116,8/26/2025 22:57:26,Student 117,Perempuan,2025,SMK,Teknik Otomasi Industri
117,8/27/2025 21:11:29,Student 118,Perempuan,2024,SMA,ips
118,8/30/2025 9:32:55,Student 119,Perempuan,2025,SMA,IPS
119,8/30/2025 9:42:34,Student 120,Perempuan,2025,SMA,IPS


# 3 | Checking the Data
In this part, I check the data whether it contains any missing values, duplicates, outliers, and inconsistent data points.

## 3.1 Column Overview
All columns seem to be identified as "object". It would be more appropriate if they are converted to "category". However, this change will take place later after all data has been normalized.

In [17]:
print(df.dtypes)

Timestamp                         object
Nama Lengkap                      object
Jenis Kelamin                     object
Lulus dari SMP                    object
Lanjut ke Sekolah Menengah ...    object
Jurusan                           object
dtype: object


## 3.2 Checking for missing values
Status: **OK**

In [19]:
df.isna().sum(axis=0)

Timestamp                         0
Nama Lengkap                      0
Jenis Kelamin                     0
Lulus dari SMP                    0
Lanjut ke Sekolah Menengah ...    0
Jurusan                           0
dtype: int64

## 3.3 Checking for Duplicates
Checking for duplicate data is based on the primary key only. Thus, I check for duplicate on the col 'Nama Lengkap'.\
Status: **OK**

In [21]:
df['Nama Lengkap'].duplicated().sum()

0

## 3.4 Checking for outliers
This stage checks 'Jenis Kelamin', 'Lulus dari SMP', 'Lanjut ke Sekolah Menengah ...', and 'Jurusan'.

In [23]:
df['Jenis Kelamin'].value_counts()

Jenis Kelamin
Perempuan    72
Laki-laki    49
Name: count, dtype: int64

Jenis Kelamin: **OK**

In [25]:
df['Lulus dari SMP'].value_counts()

Lulus dari SMP
2025            55
2024            46
2023             7
2024-2025        4
2022             2
Tahun 2025       1
2023/2024        1
12 Juni 2025     1
thn 2025         1
cth:2024         1
tahun 2024       1
20223            1
Name: count, dtype: int64

> In this column, data must be only in one year. Other kinds of data must be converted to one year format. For instance, '2023/2024' must be changed to '2024'.\
In this case, it is more efficient to create a dictionary and change the values accordingly.

In [27]:
normalize_Lulus_dari_SMP = {'2024-2025': '2025',
                           'Tahun 2025': '2025',
                           '2023/2024': '2024',
                           '12 Juni 2025': '2025',
                           'thn 2025': '2025',
                           'cth:2024': '2024',
                           'tahun 2024': '2024',
                           '20223': '2023'}
df['Lulus dari SMP'].map(normalize_Lulus_dari_SMP).fillna(df['Lulus dari SMP']).value_counts() # checks for successful operation and prevents data loss
df['Lulus dari SMP'] = df['Lulus dari SMP'].map(normalize_Lulus_dari_SMP).fillna(df['Lulus dari SMP'])
df['Lulus dari SMP'].value_counts()

Lulus dari SMP
2025    62
2024    49
2023     8
2022     2
Name: count, dtype: int64

Lulus dari SMP: **OK**

In [29]:
df['Lanjut ke Sekolah Menengah ...'].value_counts()

Lanjut ke Sekolah Menengah ...
SMK                 76
SMA                 44
SMAN 1 JALAKSANA     1
Name: count, dtype: int64

> Seems like there is one outlier in col 'Lanjut ke Sekolah Menengah ...'.\
I located the specific outlier first, then change it to SMA.

In [31]:
print(df[df['Lanjut ke Sekolah Menengah ...'] == 'SMAN 1 JALAKSANA'])

             Timestamp Nama Lengkap Jenis Kelamin Lulus dari SMP  \
31  8/25/2025 13:15:29   Student 32     Laki-laki           2024   

   Lanjut ke Sekolah Menengah ... Jurusan  
31               SMAN 1 JALAKSANA     IPA  


In [33]:
df.loc[df['Lanjut ke Sekolah Menengah ...'] == 'SMAN 1 JALAKSANA', 'Lanjut ke Sekolah Menengah ...'] = 'SMA' # checkpoint
df['Lanjut ke Sekolah Menengah ...'].value_counts()

Lanjut ke Sekolah Menengah ...
SMK    76
SMA    45
Name: count, dtype: int64

Lanjut ke Sekolah Menengah ...: **OK**

In [35]:
df['Jurusan'].unique()

array(['Agribisnis Tanaman Pangan Dan Hortikultura (ATPH)', 'IPS',
       'Teknik Otomasi Industri', 'Teknik kendaraan ringan',
       'USAHA LAYANAN PARIWISATA', 'AKUNTANSI',
       'Pengembangan Perangkat Lunak dan Gim',
       'desain permodelan dan informasi bangunan',
       'Desain Pemodelan dan Informasi Bangunan', 'Teknik Audio Video',
       'ips', 'Teknik Komputer Jaringan', 'IPA',
       'Desain Komunikasi Visual',
       'Teknik Ketenagalistrikan/Teknik instalasi Tenaga Listrik (TITL)',
       'TSM (TEKNIK SEPEDA MOTOR)', 'Teknik Kontruksi Dan Perumahan',
       'TKP (tehnik konstruksi dan perumahan)',
       'Akuntansi keuangan dan lembaga', 'teknik otomotif',
       'tkj (teknik komputer dan jaringan)',
       'SMK: Teknik kendaraan Ringan', 'teknik audio video',
       'jurusan TSM', 'jurusan teknik permesinan',
       'JURUSAN PEMESINAN (TPM)', 'Teknik Audio Vidio',
       'Teknik Sepeda Motor', 'teknik permesinan smk karya nasional',
       'TEKNIK KENDARAAN RINGAN', '

> This column seems to be taking a like to *diversity*. The number of inconsistencies is staggering. So, I will clean this mess by making a dictionary. To speed up things, I use Generative AI such as Grok 4, Gemini 2.5 Pro, and ChatGPT. The use of three distinct generative AIs serve as a cross-check, whether any of them made mistake.

In [180]:
dictionary_jurusan = {
    # IPA / IPS
    "ipa": "Ilmu Pengetahuan Alam",
    "sma: ipa": "Ilmu Pengetahuan Alam",
    "sma, ipa": "Ilmu Pengetahuan Alam",
    "sma:ipa": "Ilmu Pengetahuan Alam",
    "IPS": "Ilmu Pengetahuan Sosial",
    "ips": "Ilmu Pengetahuan Sosial",
    "SMA: IPA": "Ilmu Pengetahuan Alam",
    "SMAN 1 CILIMUS = jurusan peminatan Matematika Tingkat Lanjut,Biologi,Ekonomi": "Ilmu Pengetahuan Alam",
    "Ips": "Ilmu Pengetahuan Sosial",
    "SMA, IPA": "Ilmu Pengetahuan Alam",
    "SMA: IPA": "Ilmu Pengetahuan Alam",
    "IPA": "Ilmu Pengetahuan Alam",
    "SMA:IPA": "Ilmu Pengetahuan Alam",
    

    # Rekayasa Perangkat Lunak
    "rekayasa perangkat lunak": "Rekayasa Perangkat Lunak",
    "pengembangan perangkat lunak dan gim": "Pengembangan Perangkat Lunak dan Gim",
    "rpl": "Rekayasa Perangkat Lunak",
    "Rekayasa perangkat lunak": "Rekayasa Perangkat Lunak",
    "REKAYASA PERANGKAT LUNAK": "Rekayasa Perangkat Lunak",

    # Desain Komunikasi Visual
    "desain komunikasi visual": "Desain Komunikasi Visual",
    "dkv": "Desain Komunikasi Visual",
    "DESAIN KOMUNIKASI VISUAL": "Desain Komunikasi Visual",
    "SMKN 1 KUNINGAN : Desain komunikasi visual (DKV": "Desain Komunikasi Visual",
    "Desain komunikasi Visual": "Desain Komunikasi Visual",

    # Desain Pemodelan dan Informasi Bangunan
    "desain pemodelan dan informasi bangunan": "Desain Pemodelan dan Informasi Bangunan",
    "desain permodelan dan informasi bangunan": "Desain Pemodelan dan Informasi Bangunan",
    "dpib": "Desain Pemodelan dan Informasi Bangunan",
    "Desain Permodelan dan Informasi Bangunan": "Desain Pemodelan dan Informasi Bangunan",

    # Teknik Otomasi Industri
    "teknik otomasi industri": "Teknik Otomasi Industri",
    "toi": "Teknik Otomasi Industri",
    "SMKN 3 KUNINGAN : TEKNIK OTAMASI INDUSTRI": "Teknik Otomasi Industri",
    "SMK N 3 KUNINGAN (TOI)": "Teknik Otomasi Industri",

    # Teknik Kendaraan Ringan
    "teknik kendaraan ringan": "Teknik Kendaraan Ringan",
    "teknik kendaraan ringgan": "Teknik Kendaraan Ringan",
    "smk: teknik kendaraan ringan": "Teknik Kendaraan Ringan",
    "teknik kendaraan ringan (tkr)": "Teknik Kendaraan Ringan",
    "tkr": "Teknik Kendaraan Ringan",
    "SMK: Teknik kendaraan Ringan": "Teknik Kendaraan Ringan",
    "TEKNIK KENDARAAN RINGAN": "Teknik Kendaraan Ringan",
    "Teknik kendaraan ringan": "Teknik Kendaraan Ringan",

    # Teknik Sepeda Motor
    "teknik sepeda motor": "Teknik Sepeda Motor",
    "tsm": "Teknik Sepeda Motor",
    "tbsm": "Teknik dan Bisnis Sepeda Motor",
    "jurusan tsm": "Teknik Sepeda Motor",
    "teknik sepedah motor": "Teknik Sepeda Motor",
    "TSM (TEKNIK SEPEDA MOTOR)": "Teknik Sepeda Motor",
    "jurusan TSM": "Teknik Sepeda Motor",
    "TBSM": "Teknik dan Bisnis Sepeda Motor",
    "TEKNIK SEPEDA MOTOR": "Teknik Sepeda Motor",
    "Teknik sepeda motor": "Teknik Sepeda Motor",

    # Teknik Komputer Jaringan
    "teknik komputer jaringan": "Teknik Komputer dan Jaringan",
    "teknik komputer dan jaringan": "Teknik Komputer dan Jaringan",
    "tkj": "Teknik Komputer dan Jaringan",
    "tjk": "Teknik Komputer dan Jaringan",
    "teknik komputar jaringan": "Teknik Komputer dan Jaringan",
    "tkj (teknik komputer dan jaringan)": "Teknik Komputer dan Jaringan",
    "Teknik Komputar Jaringan": "Teknik Komputer dan Jaringan",
    "Teknik Komputer dan Jaringan (TKJ)": "Teknik Komputer dan Jaringan",
    "Teknik komputer dan jaringan": "Teknik Komputer dan Jaringan",
    "Kelas khusus Axioo class program": "Teknik Komputer dan Jaringan",

    # Teknik Audio Video
    "teknik audio video": "Teknik Audio Video",
    "teknik audio vidio": "Teknik Audio Video",
    "Teknik Audio Vidio": "Teknik Audio Video",

    # Teknik Pemesinan
    "teknik permesinan": "Teknik Pemesinan",
    "jurusan teknik permesinan": "Teknik Pemesinan",
    "jurusan pemesinan (tpm)": "Teknik Pemesinan",
    "teknik permesinan smk karya nasional": "Teknik Pemesinan",
    "JURUSAN PEMESINAN (TPM)": "Teknik Permesinan",
    "Teknik pemesinan": "Teknik Permesinan",

    # Teknik Ketenagalistrikan / Instalasi Listrik
    "teknik ketenagalistrikan": "Teknik Instalasi Tenaga Listrik",
    "teknik instalasi tenaga listrik": "Teknik Instalasi Tenaga Listrik",
    "titl": "Teknik Instalasi Tenaga Listrik",
    "Teknik Ketenagalistrikan/Teknik instalasi Tenaga Listrik (TITL)": "Teknik Instalasi Tenaga Listrik",
    "Teknik Ketenagalistrikan": "Teknik Instalasi Tenaga Listrik",

    # Teknik Konstruksi Perumahan
    "teknik kontruksi dan perumahan": "Teknik Konstruksi dan Perumahan",
    "tkp": "Teknik Konstruksi dan Perumahan",
    "TKP (tehnik konstruksi dan perumahan)": "Teknik Konstruksi dan Perumahan",

    # Akuntansi
    "akuntansi": "Akuntansi",
    "akuntansi keuangan dan lembaga": "Akuntansi dan Keuangan Lembaga",
    "akuntansi dan keuangan lembaga": "Akuntansi dan Keuangan Lembaga",
    "AKUNTANSI": "Akuntansi",
    "Akuntansi keuangan dan lembaga": "Akuntansi dan Keuangan Lembaga",

    # Pemasaran
    "pemasaran": "Pemasaran",
    "jurusan pemasaran": "Pemasaran",
    "pemasaran / bisnis ritel": "Pemasaran",
    "Jurusan pemasaran": "Pemasaran",
    "PEMASARAN / BISNIS RITEL": "Pemasaran",

    # Perhotelan
    "perhotelan": "Perhotelan",

    # Tata Boga
    "tata boga": "Tata Boga",

    # Farmasi
    "farmasi": "Farmasi",
    "smk (kefarmasian)": "Farmasi",
    "SMK (kefarmasian)": "Farmasi",

    # Keperawatan
    "keperawatan": "Keperawatan",
    "jurusan keperawatan": "Keperawatan",

    # Agribisnis
    "Agribisnis Tanaman Pangan Dan Hortikultura (ATPH)": "Agribisnis Tanaman Pangan dan Hortikultura",
    "agribisnis tanaman pangan dan hortikultura": "Agribisnis Tanaman Pangan dan Hortikultura",
    "atph": "Agribisnis Tanaman Pangan dan Hortikultura",
    "agribisnis perikanan air tawar": "Agribisnis Perikanan Air Tawar",

    # Usaha Layanan Pariwisata
    "usaha layanan pariwisata": "Usaha Layanan Pariwisata",
    "USAHA LAYANAN PARIWISATA": "Usaha Layanan Pariwisata",

    # Layanan Perbankan
    "layanan perbankan": "Layanan Perbankan",

    # Teknik Otomotif
    "teknik otomotif": "Teknik Otomotif"
}
dictionary_jurusan

{'ipa': 'Ilmu Pengetahuan Alam',
 'sma: ipa': 'Ilmu Pengetahuan Alam',
 'sma, ipa': 'Ilmu Pengetahuan Alam',
 'sma:ipa': 'Ilmu Pengetahuan Alam',
 'IPS': 'Ilmu Pengetahuan Sosial',
 'ips': 'Ilmu Pengetahuan Sosial',
 'SMA: IPA': 'Ilmu Pengetahuan Alam',
 'SMAN 1 CILIMUS = jurusan peminatan Matematika Tingkat Lanjut,Biologi,Ekonomi': 'Ilmu Pengetahuan Alam',
 'Ips': 'Ilmu Pengetahuan Sosial',
 'SMA, IPA': 'Ilmu Pengetahuan Alam',
 'IPA': 'Ilmu Pengetahuan Alam',
 'SMA:IPA': 'Ilmu Pengetahuan Alam',
 'rekayasa perangkat lunak': 'Rekayasa Perangkat Lunak',
 'pengembangan perangkat lunak dan gim': 'Pengembangan Perangkat Lunak dan Gim',
 'rpl': 'Rekayasa Perangkat Lunak',
 'Rekayasa perangkat lunak': 'Rekayasa Perangkat Lunak',
 'REKAYASA PERANGKAT LUNAK': 'Rekayasa Perangkat Lunak',
 'desain komunikasi visual': 'Desain Komunikasi Visual',
 'dkv': 'Desain Komunikasi Visual',
 'DESAIN KOMUNIKASI VISUAL': 'Desain Komunikasi Visual',
 'SMKN 1 KUNINGAN : Desain komunikasi visual (DKV': 'Desain

In [182]:
df['Jurusan'] = df['Jurusan'].replace(dictionary_jurusan)

In [184]:
df['Jurusan'].unique()

array(['Agribisnis Tanaman Pangan dan Hortikultura',
       'Ilmu Pengetahuan Sosial', 'Teknik Otomasi Industri',
       'Teknik Kendaraan Ringan', 'Usaha Layanan Pariwisata', 'Akuntansi',
       'Pengembangan Perangkat Lunak dan Gim',
       'Desain Pemodelan dan Informasi Bangunan', 'Teknik Audio Video',
       'Teknik Komputer Jaringan', 'Ilmu Pengetahuan Alam',
       'Desain Komunikasi Visual', 'Teknik Instalasi Tenaga Listrik',
       'Teknik Sepeda Motor', 'Teknik Kontruksi Dan Perumahan',
       'Teknik Konstruksi dan Perumahan',
       'Akuntansi dan Keuangan Lembaga', 'Teknik Otomotif',
       'Teknik Komputer dan Jaringan', 'Teknik Pemesinan',
       'Teknik Permesinan', 'Teknik dan Bisnis Sepeda Motor',
       'Keperawatan', 'Farmasi', 'Agribisnis Perikanan Air Tawar',
       'Rekayasa Perangkat Lunak', 'Layanan Perbankan', 'Perhotelan',
       'Tata boga', 'Pemasaran'], dtype=object)

In [186]:
df['Jurusan'].value_counts()

Jurusan
Ilmu Pengetahuan Alam                         25
Ilmu Pengetahuan Sosial                       20
Teknik Sepeda Motor                            9
Teknik Komputer dan Jaringan                   7
Desain Pemodelan dan Informasi Bangunan        5
Desain Komunikasi Visual                       5
Teknik Otomasi Industri                        4
Teknik Kendaraan Ringan                        4
Teknik Audio Video                             4
Pemasaran                                      3
Keperawatan                                    3
Akuntansi dan Keuangan Lembaga                 3
Teknik Instalasi Tenaga Listrik                3
Pengembangan Perangkat Lunak dan Gim           3
Rekayasa Perangkat Lunak                       2
Teknik Komputer Jaringan                       2
Akuntansi                                      2
Teknik Pemesinan                               2
Teknik Permesinan                              2
Teknik dan Bisnis Sepeda Motor                 2
Farmasi     

In [188]:
df

,Timestamp,Nama Lengkap,Jenis Kelamin,Lulus dari SMP,Lanjut ke Sekolah Menengah ...,Jurusan
0,8/25/2025 9:45:19,Student 1,Laki-laki,2025,SMK,Agribisnis Tanaman Pangan dan Hortikultura
1,8/25/2025 9:51:54,Student 2,Laki-laki,2025,SMA,Ilmu Pengetahuan Sosial
2,8/25/2025 9:57:09,Student 3,Laki-laki,2025,SMK,Teknik Otomasi Industri
3,8/25/2025 10:05:45,Student 4,Laki-laki,2025,SMK,Teknik Kendaraan Ringan
4,8/25/2025 10:08:43,Student 5,Perempuan,2025,SMK,Usaha Layanan Pariwisata
...,...,...,...,...,...,...
116,8/26/2025 22:57:26,Student 117,Perempuan,2025,SMK,Teknik Otomasi Industri
117,8/27/2025 21:11:29,Student 118,Perempuan,2024,SMA,Ilmu Pengetahuan Sosial
118,8/30/2025 9:32:55,Student 119,Perempuan,2025,SMA,Ilmu Pengetahuan Sosial
119,8/30/2025 9:42:34,Student 120,Perempuan,2025,SMA,Ilmu Pengetahuan Sosial


> Export df to CSV with name "for-data-vis"

In [190]:
df.to_csv('for-data-vis.csv', index=False)

## Numerical Outlier

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Timestamp                       121 non-null    object
 1   Nama Lengkap                    121 non-null    object
 2   Jenis Kelamin                   121 non-null    object
 3   Lulus dari SPENSANA  ...        121 non-null    object
 4   Lanjut ke Sekolah Menengah ...  121 non-null    object
 5   Jurusan ...                     121 non-null    object
dtypes: object(6)
memory usage: 5.8+ KB


In [20]:
for col in df.columns:
    df[col] = df[col].astype('category')

print(df.dtypes)

Timestamp                         category
Nama Lengkap                      category
Jenis Kelamin                     category
Lulus dari SMP                    category
Lanjut ke Sekolah Menengah ...    category
Jurusan                           category
dtype: object
